In [1]:
# Francisco Dominguez Mateos
# 07/09/2021
# Williams Sindrome detection of: disease, gender, identity, age

In [2]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import random
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout, Flatten
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [3]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc


def shuffle_list(*ls):
  l =list(zip(*ls))
  random.shuffle(l)
  return zip(*l)

DISEASE_NAME="SW"
#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA_F="/home/spika/datasets/sonido/voz/diseases/sw/asr_sw_f_21090701"
PATH_DATA_M="/home/spika/datasets/sonido/voz/diseases/sw/asr_sw_m_21090701"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    def __init__(self):
        '''
        Constructor
        '''
        self.max_pad_len=44
        self.nval=20
        basepathF=PATH_DATA_F
        basepathM=PATH_DATA_M
        self.mfccs_n=[]
        self.speakers_n=[]
        self.diseases_n=[]
        self.ages_n=[]
        self.genders_n=[]
        self.mfccs_d=[]
        self.speakers_d=[]
        self.diseases_d=[]
        self.ages_d=[]
        self.genders_d=[]
        fNamesF=os.listdir(basepathF)
        print("fNamesF=",len(fNamesF))
        random.shuffle(fNamesF)
        for i,fn in enumerate(fNamesF):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!=DISEASE_NAME and disease!="N": print("wrong disease label=",disease)
            if disease!=DISEASE_NAME: continue
            mfccs=wav2mfcc(basepathF+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
            self.genders_d.append("F")
        fNamesM=os.listdir(basepathM)
        print("fNamesM=",len(fNamesM))
        random.shuffle(fNamesM)
        for i,fn in enumerate(fNamesM):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!=DISEASE_NAME and disease!="N": print("wrong disease label=",disease)
            if disease!=DISEASE_NAME: continue
            mfccs=wav2mfcc(basepathM+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
            self.genders_d.append("M")
        nSMS=len(self.mfccs_d)
        print("n",DISEASE_NAME,"=",nSMS)
        #SHUFFLE
        self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d=shuffle_list(self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d)
        self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d=list(self.mfccs_d),list(self.speakers_d),list(self.diseases_d),list(self.ages_d),list(self.genders_d)

        random.shuffle(fNamesF)
        random.shuffle(fNamesM)
        for i,fn in enumerate(fNamesF):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepathF+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            self.genders_n.append("F")
            idxSMS=random.randint(self.nval,nSMS)#in order to not repeat validation samples
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])
            self.genders_d.append(self.genders_d[idxSMS])
        for i,fn in enumerate(fNamesM):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepathM+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            self.genders_n.append("M")
            idxSMS=random.randint(self.nval,nSMS)#in order to not repeat validation samples
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])
            self.genders_d.append(self.genders_d[idxSMS])
        #SHUFFLE
        self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n=shuffle_list(self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n)
        self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n=list(self.mfccs_n),list(self.speakers_n),list(self.diseases_n),list(self.ages_n),list(self.genders_n)

        self.mfccs_val   =self.mfccs_d   [:self.nval]+self.mfccs_n   [:self.nval]
        self.speakers_val=self.speakers_d[:self.nval]+self.speakers_n[:self.nval]
        self.diseases_val=self.diseases_d[:self.nval]+self.diseases_n[:self.nval]
        self.ages_val    =self.ages_d    [:self.nval]+self.ages_n    [:self.nval]
        self.genders_val =self.genders_d [:self.nval]+self.genders_n [:self.nval]

        self.mfccs   =self.mfccs_d   [nSMS+self.nval:]+self.mfccs_n   [self.nval:]
        self.speakers=self.speakers_d[nSMS+self.nval:]+self.speakers_n[self.nval:]
        self.diseases=self.diseases_d[nSMS+self.nval:]+self.diseases_n[self.nval:]
        self.ages    =self.ages_d    [nSMS+self.nval:]+self.ages_n    [self.nval:]
        self.genders =self.genders_d [nSMS+self.nval:]+self.genders_n [self.nval:]

        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers+self.speakers_val)
        self.iSpeakerLabels    =self.leSpeakers.transform(self.speakers)
        self.iSpeakerLabels_val=self.leSpeakers.transform(self.speakers_val)
        
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases+self.diseases_val)
        self.iDiseaseLabels    =self.leDiseases.transform(self.diseases)
        self.iDiseaseLabels_val=self.leDiseases.transform(self.diseases_val)
        
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages+self.ages_val)
        self.iAgeLabels    =self.leAges.transform(self.ages)
        self.iAgeLabels_val=self.leAges.transform(self.ages_val)
        
        self.leGenders=prep.LabelEncoder()
        self.leGenders.fit(self.genders+self.genders_val)
        self.iGendersLabels    =self.leGenders.transform(self.genders)
        self.iGendersLabels_val=self.leGenders.transform(self.genders_val)
        
        print("Samples=",len(self.diseases))
        print(DISEASE_NAME,"=",self.diseases.count(DISEASE_NAME))
        print("N   =",self.diseases.count('N'))
        print(set(self.diseases))
        print(set(self.ages))
        print(set(self.speakers))
        print(set(self.genders))
        print(len(self.diseases))
        print(len(self.genders))
        print(len(self.iGendersLabels))
        print(len(self.iGendersLabels_val))

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNamesF= 762
fNamesM= 924
n SW = 109
Samples= 3114
SW = 1557
N   = 1557
{'SW', 'N'}
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 21, 23, 27, 31}
{'STA', 'BMM', 'HXA', 'JCL', 'WZA', 'BHH', 'MVD', 'VRP', 'CMN', 'MPC', 'MCS', 'ITA', 'CFM', 'GOM', 'LPM', 'AMC', 'PRV', 'AAI', 'IMA', 'RLL', 'RAA', 'NVD', 'SPI', '11C', 'YMA', 'NVA', 'FDN', 'LPD', 'MRA', 'BBA', 'MXS', 'PMA', 'CPM', '10V', 'EMA', 'KCR', 'CFR', 'SHA', 'PGA', 'FMI', 'CVP', 'MAJ', 'CRO', 'ROJ', 'GDA', 'ZGz', 'LFJ', 'BPC', 'ERA', 'EMG', 'CCM', 'TVD', 'SBA', 'MMD', 'ABN', 'CRD', 'LMS', 'WMJ', 'hMA', 'CFE', 'RSA', 'PRE', 'MJM', 'CLR', 'MRR', 'VSY', 'MLM', '10J', 'BCA', 'RZA', 'GPC', 'PDI', 'GBD', 'GMM', 'MCN', 'PLW', 'MMN', 'FSP', 'ZMM', '10L', 'RGA', 'ACD', 'RDA', 'SOS', 'CRI', 'MPJ', 'DMD', 'ELL', 'UOM', 'RCL', 'MCA', 'PMP', 'MDS', 'SCI', 'LSC', 'IVM', 'PLD', 'BFA', 'RLJ', 'SCL', 'MGH', 'YZS', 'ICJ', 'CII', 'FDS', 'PMV', 'VPB', 'MOA', 'BCN', 'MAP', 'PGM', 'PPM', 'MTA', 'PTA', 'EIR', 'RLD', 'TDM', 'LDM', 'HGR', 'MMF', 'RO

In [4]:
trainX, trainYdiseases, trainYspeakers, trainYages, trainYgenders = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels),np.array(ea.iGendersLabels)
#testX, testY = test
valX,   valYdiseases,   valYspeakers,   valYages,   valYgenders   = np.array(ea.mfccs_val),np.array(ea.iDiseaseLabels_val),np.array(ea.iSpeakerLabels_val),np.array(ea.iAgeLabels_val),np.array(ea.iGendersLabels_val)

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=len(ea.leDiseases.classes_)
outputSpeakers_dim=len(ea.leSpeakers.classes_)
outputAges_dim    =len(ea.leAges.classes_)
outputGenders_dim =len(ea.leGenders.classes_)

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
trainYgenders_cat  = utils.to_categorical(trainYgenders ,outputGenders_dim)
print(len(ea.iGendersLabels),3124-2276)
print(ea.iGendersLabels)
print(trainYgenders.shape)
print(trainYgenders_cat.shape)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape,trainYgenders_cat.shape)

# Converting labels to binary vectors
valYdiseases_cat = utils.to_categorical(valYdiseases,outputDiseases_dim)
valYspeakers_cat = utils.to_categorical(valYspeakers,outputSpeakers_dim)
valYages_cat     = utils.to_categorical(valYages    ,outputAges_dim)
valYgenders_cat  = utils.to_categorical(valYgenders ,outputGenders_dim)
print (valX.shape,valYdiseases_cat.shape,valYspeakers_cat.shape,valYages_cat.shape,valYgenders_cat.shape)

2 335 16 2
3114 848
[0 0 0 ... 0 1 0]
(3114,)
(3114, 2)
(3114, 44, 20) (3114, 2) (3114, 335) (3114, 16) (3114, 2)
(40, 44, 20) (40, 2) (40, 335) (40, 16) (40, 2)


In [27]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim,hiden_dim=128):
    DROPOUT_RATE=0.55
    input = Input(shape=(ea.max_pad_len,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    x = Dropout(DROPOUT_RATE)(x)
    print(x.shape)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = Dropout(DROPOUT_RATE)(x)
    print(x.shape)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    #x = Dropout(DROPOUT_RATE)(x)
    #print(x.shape)
    #x = Conv1D(hiden_dim*2, 3, padding="valid", activation="relu", strides=2)(x)
    #x = Dropout(DROPOUT_RATE)(x)
    #print(x.shape)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    #x = Flatten()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    wd= Dense(hiden_dim, activation="relu")(x)
    sd= Dense(hiden_dim, activation="relu")(x)
    ad= Dense(hiden_dim, activation="relu")(x)
    gd= Dense(hiden_dim, activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)
    prediction_gender   = Dense(outputGenders_dim , activation="softmax", name="gender" )(gd)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages,prediction_gender])
    return model

In [28]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 44, 20)
(None, 42, 128)
(None, 40, 128)
(None, 128)
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 44, 20)]     0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 42, 128)      7808        input_6[0][0]                    
__________________________________________________________________________________________________
dropout_20 (Dropout)            (None, 42, 128)      0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 40, 128)      49280       dropout_20[0][0]                 
____________________

In [29]:
checkpoint_filepath="checkpoints/sms/sw"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_disease_accuracy',
    mode='max',
    save_best_only=True)

In [30]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat,trainYgenders_cat], batch_size=100,epochs=150,validation_data=(valX, [valYdiseases_cat,valYspeakers_cat,valYages_cat,valYgenders_cat]))

Epoch 1/150
32/32 [==============================] - 2s 52ms/step - loss: 33.1964 - disease_loss: 5.2118 - speaker_loss: 13.2626 - age_loss: 11.5715 - gender_loss: 3.1506 - disease_accuracy: 0.4939 - speaker_accuracy: 0.0180 - age_accuracy: 0.0896 - gender_accuracy: 0.5267 - val_loss: 9.9457 - val_disease_loss: 0.7231 - val_speaker_loss: 5.7649 - val_age_loss: 2.7495 - val_gender_loss: 0.7082 - val_disease_accuracy: 0.5000 - val_speaker_accuracy: 0.0000e+00 - val_age_accuracy: 0.1000 - val_gender_accuracy: 0.3750
Epoch 2/150
32/32 [==============================] - 1s 37ms/step - loss: 9.8977 - disease_loss: 0.7157 - speaker_loss: 5.7410 - age_loss: 2.7453 - gender_loss: 0.6957 - disease_accuracy: 0.5148 - speaker_accuracy: 0.0222 - age_accuracy: 0.1349 - gender_accuracy: 0.5642 - val_loss: 9.8869 - val_disease_loss: 0.7039 - val_speaker_loss: 5.7481 - val_age_loss: 2.7103 - val_gender_loss: 0.7246 - val_disease_accuracy: 0.4000 - val_speaker_accuracy: 0.0250 - val_age_accuracy: 0.0750

In [74]:
idxSample=288
print(trainX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge,predGender=model.predict(trainX[idxSample:idxSample+1,:,:])
#print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
y_predGender   = np.argmax(predGender)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
print(y_predAge,"="    ,np.max(predGender))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
gender =    ea.leGenders.inverse_transform([y_predGender])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print("gender =",ea.genders[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)
print(ea.leGenders.classes_)

(1, 44, 20)
1 = 0.9987111
34 = 0.99999213
3 = 0.9998969
3 = 0.99999726
Predictions:
BCN  disease is ' SW ' with age= 6
Groundtruth:
speaker= BCN
disease= SW
ages   = 6
gender = M

['N' 'SW']
['10C' '10I' '10J' '10L' '10S' '10V' '11C' '11P' 'AAE' 'AAI' 'AAJ' 'ABN'
 'ACA' 'ACD' 'ACI' 'ACR' 'ADD' 'ADS' 'AFE' 'AIE' 'AIO' 'AIP' 'AMA' 'AMC'
 'API' 'ARM' 'ASC' 'ASN' 'AZM' 'BBA' 'BBE' 'BBR' 'BCA' 'BCB' 'BCN' 'BDC'
 'BFA' 'BFR' 'BGE' 'BHH' 'BMM' 'BPC' 'BTS' 'CAA' 'CAR' 'CCC' 'CCM' 'CCN'
 'CFE' 'CFM' 'CFR' 'CGA' 'CGC' 'CGJ' 'CII' 'CLL' 'CLR' 'CMA' 'CMI' 'CMN'
 'CMS' 'CPA' 'CPD' 'CPM' 'CRC' 'CRD' 'CRE' 'CRI' 'CRO' 'CSA' 'CSJ' 'CSR'
 'CTC' 'CVP' 'DAP' 'DDR' 'DFM' 'DHD' 'DLA' 'DMD' 'DMH' 'DPN' 'DRI' 'DRS'
 'EIR' 'ELL' 'EMA' 'EMC' 'EMG' 'EMP' 'EPS' 'ERA' 'ERJ' 'FCD' 'FCL' 'FDN'
 'FDS' 'FEA' 'FFR' 'FGI' 'FGJ' 'FLS' 'FMI' 'FOV' 'FRM' 'FSM' 'FSP' 'GAE'
 'GBA' 'GBD' 'GDA' 'GDT' 'GES' 'GLA' 'GMD' 'GMM' 'GOM' 'GPC' 'GRA' 'GSA'
 'GSP' 'HAM' 'HCL' 'HGC' 'HGH' 'HGR' 'HJS' 'HMF' 'HMJ' 'HPP' 'HSI' 'HXA'
 'ICJ'

In [75]:
idxSample=26
print(valX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge,predGender=model.predict(valX[idxSample:idxSample+1,:,:])
#print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
y_predGender   = np.argmax(predGender)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
print(y_predAge,"="    ,np.max(predGender))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
gender =    ea.leGenders.inverse_transform([y_predGender])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print("gender =",ea.genders[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)
print(ea.leGenders.classes_)

(1, 44, 20)
0 = 0.99851805
78 = 0.030375764
4 = 0.23920295
4 = 0.5921034
Predictions:
DLA  disease is ' N ' with age= 7
Groundtruth:
speaker= BMM
disease= SW
ages   = 23
gender = F

['N' 'SW']
['10C' '10I' '10J' '10L' '10S' '10V' '11C' '11P' 'AAE' 'AAI' 'AAJ' 'ABN'
 'ACA' 'ACD' 'ACI' 'ACR' 'ADD' 'ADS' 'AFE' 'AIE' 'AIO' 'AIP' 'AMA' 'AMC'
 'API' 'ARM' 'ASC' 'ASN' 'AZM' 'BBA' 'BBE' 'BBR' 'BCA' 'BCB' 'BCN' 'BDC'
 'BFA' 'BFR' 'BGE' 'BHH' 'BMM' 'BPC' 'BTS' 'CAA' 'CAR' 'CCC' 'CCM' 'CCN'
 'CFE' 'CFM' 'CFR' 'CGA' 'CGC' 'CGJ' 'CII' 'CLL' 'CLR' 'CMA' 'CMI' 'CMN'
 'CMS' 'CPA' 'CPD' 'CPM' 'CRC' 'CRD' 'CRE' 'CRI' 'CRO' 'CSA' 'CSJ' 'CSR'
 'CTC' 'CVP' 'DAP' 'DDR' 'DFM' 'DHD' 'DLA' 'DMD' 'DMH' 'DPN' 'DRI' 'DRS'
 'EIR' 'ELL' 'EMA' 'EMC' 'EMG' 'EMP' 'EPS' 'ERA' 'ERJ' 'FCD' 'FCL' 'FDN'
 'FDS' 'FEA' 'FFR' 'FGI' 'FGJ' 'FLS' 'FMI' 'FOV' 'FRM' 'FSM' 'FSP' 'GAE'
 'GBA' 'GBD' 'GDA' 'GDT' 'GES' 'GLA' 'GMD' 'GMM' 'GOM' 'GPC' 'GRA' 'GSA'
 'GSP' 'HAM' 'HCL' 'HGC' 'HGH' 'HGR' 'HJS' 'HMF' 'HMJ' 'HPP' 'HSI' 'HXA'
 'IC